 - Diana Zaray Corado #191025
 - Pablo Alejandro Méndez #19195
- Orlando Osberto Cabrera #19943
# Hoja de Trabajo 7 - Redes Neuronales Artificales  

In [ ]:
# Librerias
import pandas as pd
import numpy as np
import scipy.stats as sp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
# General variables
cuantitative = [
    'LotFrontage',
    'LotArea',
    'MiscVal',
	'WoodDeckSF',
    'OpenPorchSF',
    'EnclosedPorch',
    '3SsnPorch',
    'ScreenPorch',
    'PoolArea',
    'GarageArea',
    'GrLivArea',
    'LowQualFinSF',
    '2ndFlrSF',
    '1stFlrSF',
    'TotalBsmtSF',
    'BsmtUnfSF',
    'BsmtFinSF2',
    'BsmtFinSF1',
    'MasVnrArea',
    'BsmtFullBath',
    'BsmtHalfBath',
    'FullBath',
    'HalfBath',
    'KitchenAbvGr',
    'TotRmsAbvGrd',
    'Fireplaces',
    'GarageCars',
    'SalePrice',
]

# Exploración de los datos
Para el desarrollo de modelos de clasificación y de regresión lineal se cuenta con un conjunto de datos provisto por Kaggle, la cual es una comunidad en línea de científicos de datos, que permite encontrar conjuntos de datos para explorar y construir modelos. El data set a utilizar se conoce como *House Prices: Advance Regression Techniques*, el cual tiene tanto un conjunto de entrenamiento como un conjunto de prueba. En esta hoja se estara utilizando el conjunto de entrenamiento, que cuenta con 81 variables y 1460 observaciones, el cual a su vez un 70% de los datos se utilizará como entrenamiento y un 30% como prueba. 

Se omite el análisis exploratorio del conjunto de datos debido a que ya se ha presentado en hojas de trabajo anteriores 

In [ ]:
train = pd.read_csv("./train.csv")
train_shape = train.shape

# Resumen de los datos
head = train.head().style.set_properties(**{'text-align': 'center'}) 
display(head)
del head

## Preprocesamiento
Para el preprocesamiento de los datos se procede a validar si existen observaciones con valores faltantes y se realiza un escalamentiento de los datos.

In [ ]:
select_train = train[cuantitative]
# Asegurando que no existan valores nan o inf
select_train = select_train[~select_train.isin([np.nan, np.inf, -np.inf]).any(1)]

Por análisis realizados en hojas anteriores se sabe que los datos no siguen una distribución normal, por lo tal es necesario realizar una normalización o una estandarización para garantizar que ninguna variable que tenga rango de valores muy grandes sean más influeyentes en el modelo que aquellas con rango de valores más bajos. Sin embargo, para poder agregar la variable clasificadora, la estandarización se realizará antes de hacer la separación de los conjuntos de datos en entrenamiento y prueba. 

Si bien es muy común que al momento de hacer preprocesamiento de los datos se realice tratamiento de outliers, las redes neuronales, especialmente aquellas que son multicapa, debido a que es un proceso de varios pasos para hacer el *fit* a los datos, esto abre paso a que el modelo sea más flexible y disminuya el impacto de los outliers. Por otro lado, generalmente las funciones de activación, proveen una especia de "aplanamiento" lo cual le da la capacidad al modelo de tratar los *outliers* y de que estos no tengan un impacto significativo en el modelo. 

Por lo mencionado anteriormente, no se realizará ningún tipo de tratamiento de *outliers* además de para garantizar una comparación equitativa con los modelos realizados en otras hojas de trabajo.

## Conjunto de prueba y entrenamiento
Seleccione como variable respuesta la que creó con las categorías del precio de la casa.

### Agregando la variable objetivo al conjunto de datos
Debido a que la variable objetivo es categórica y las redes neuronales requieren que los datos sean numéricos, es necesario codificar las categorías mediante números. Para codificar las categorías se usaran los siguientes valores:
- Baratas: 1
- Intermedias: 10
- Caras: 100

In [ ]:
# Agregando la nueva variable al data frame
conditions = [
    (select_train['SalePrice'] <= 171500),
    (select_train['SalePrice'] > 171500) & (select_train['SalePrice'] <= 295500),
    (select_train['SalePrice'] > 295500) 
    ]

values = [1, 10, 100]

select_train['HouseCategory'] = np.select(conditions, values)
del values, conditions

### Selección de variables
Para seleccionar las variables a utilizar dentro del modelo, se inició tomando en consideración únicamente las variables numéricas. Seguido a esto, se realizó una correlación entre todas las variables cuantitativas para poder analizar cuáles son las que influyen significativamente en el precio de venta y con base a esto se seleccionó el conjunto de *features* a utilizar.

In [ ]:
cuantitative_data = train[cuantitative]
correlation = cuantitative_data.corr(method = 'spearman')
plt.figure(figsize=(25,12))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, mask=matrix)
plt.show()

del correlation, cuantitative_data, matrix

Como se puede observar en la tabla y gráfica anterior se presentan aquellas variables que cuentan con una alta correlación (tomando como correlación alta a valores mayores o iguales a 0.5). A continuación se listan la correlaciones encontradas por variables:
- LotFrontage → LotArea
- GarageArea → GarageCars, SalePrice
- GrLivArea → 2ndFlrSF, FullBath, TotRmsAbvGrd, GarageCars, SalePrice
- 2ndFlrSF → HalfBath, TotRmsAbvGrd
- 1stFlrSF → TotalBsmtSF, SalePrice
- TotalBsmtSF → SalePrice
- BsmtUnfSF → BsmtFinSF1, BsmtFullBath
- FullBath → TotRmsAbvGrd, GarageCars, SalePrice
- TotRmsAbvGrd → SalePrice
- Fireplaces → SalePrice
- GargeCars → SalePrice

La estrecha correlación con la que cuentan las variables entre sí representa un potencial error para el modelo, ya que como bien se sabe, uno de los supuestos dentro del modelo de regresión logísticas es que las variables no presenten multicolinealidad ya que esto podría sesgar dicho modelo a la información "repetida" presentada por estas variables. Por lo tal, para evitar un sesgo y *overfitting* del modelo se eliminaran las variables correlacionadas, dejando solo una que represente la información de todas dentro del modelo. Las variables que se descartarán del modelo son:
- LotFrontage
- GarageCars
- TotRmsAbvGrd
- FullBath
- HalfBath
- TotalBsmtSF
- BsmtFinSF1
- BsmFullBath
- Fireplaces

In [ ]:
useless = ['LotFrontage', 
'GarageCars', 
'TotRmsAbvGrd', 
'FullBath', 
'HalfBath', 
'TotalBsmtSF', 
'BsmtFinSF1', 
'BsmtFullBath', 
'Fireplaces', 
'PoolArea', 
'LowQualFinSF', 
'BsmtFinSF2', 
'BsmtHalfBath', 
'KitchenAbvGr' ]

In [ ]:
selected_train = select_train.loc[:, ~select_train.columns.isin(useless)]

In [ ]:
# separate between target and predictors
target = selected_train.pop('HouseCategory')
predictors = selected_train.copy()

In [ ]:
# stratified sample
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, train_size  = 0.7, shuffle = True, random_state=19195)

# Clasificación
Genere dos modelos de redes neuronales que sea capaz de clasificar usando la variable respuesta que categoriza las casas en baratas, medias y caras. Estos modelos deben tener diferentes topologías y funciones de activación.

Para el primer modelo de redes neuronales se utilizaran 2 capas, la primera con 3 neuronas y la segunda con 7 neuronas, con una función de activación *hyperbolic tangent* y con un *solver adam* para la optimización de los pesos.

In [ ]:
# n general, we recommend using StandardScaler within a Pipeline in order to prevent most risks of data leaking
ht_model = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(3,5), activation='tanh', solver='adam', max_iter=1000, random_state=191943))
ht_model.fit(predictors_train, target_train)

Para el segundo modelo de redes neuronales se utilizaran 3 capas, la primera con 3 neuronas, la segunda con 5 y la tercera con 7 neuronas, con una función de activación *ReLU (Rectified Linear Unit)* y con un *solver lbfgs* para la optimización de los pesos.

In [ ]:
# n general, we recommend using StandardScaler within a Pipeline in order to prevent most risks of data leaking
relu_model = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(3,5, 7), activation='relu', solver='lbfgs', max_iter=700, random_state=191943))
relu_model.fit(predictors_train, target_train)

## Efectividad del modelo para predecir
Use los modelos para predecir el valor de la variable respuesta. Haga las matrices de confusión respectivas.

In [ ]:
target_names = ['Barata', 'Intermedia', 'Cara']

In [ ]:
# Predicción con modelo con función de activación tangente hiperbólica
prediction_ht_test = ht_model.predict(predictors_test)

In [ ]:
cm = confusion_matrix(target_test, prediction_ht_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot()
plt.show()

del cm, disp

Como se puede observar en la matriz de confusión, en general, el modelo es bastante bueno al hacer la clasificación de los datos, ya que únicamente hizo una clasificación incorrecta 5 veces. Lo interesante a notar es que hizo una clasificación perfecta de las casas tipo caras, ya que todas las casas clasificadas como caras el modelo predijo que eran caras. Por otro lado, las casas intermedias son las que el modelo clasificó de manera más incorrecta, clasificando 3 casas intermedias como caras, luego le siguen baratas, de las cuales, 2 clasificó incorrectamente como intermedias. Como bien se sabe, estos errores están asociados a la precición y el *recall* del modelo, al ser la cantidad de errores muy pequeña relativa a la cantidad de aciertos se puede decir que el modelo fue preciso en la clasificación de cada una de las categorías.

In [ ]:
# Predicción con modelo con función de activación ReLU
prediction_relu_test = relu_model.predict(predictors_test)

In [ ]:
cm = confusion_matrix(target_test, prediction_relu_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot()
plt.show()

del cm, disp

Como se puede notar en la matriz de clasificación, al igual que el modelo anterior, la categoría de casas caras fue clasificada de manera perfecta, y en general se puede observar que el modelo predice de manera muy acertada las categorías a las cuales pertenecen las observaciones. En este caso se puede observar que de las casas clasificadas como intermedias, aproximadamente un 3% fue predicha de manera incorrecta, de esto, un 2% se clasificó como casas baratas y un 1% como casas caras. Por otro lado, también se tiene que de todas las casas  baratas en el data set, aproximadamente un 2% no se predijo correctamente, ya que 3 casas caras, se predijeron como intermedias. 

Si bien como se mencionó al inicio, todas las casas clasificadas como caras el momento supo predecirlas, el modelo además predijo de forma incorrecta una casa extra como cara siendo esta intermedia. 

## Efectividad entre los modelos de redes neuronales
Compare los resultados obtenidos con los diferentes modelos de clasificación usando redes neuronales en cuanto a efectividad, tiempo de procesamiento y equivocaciones (donde el algoritmo se equivocó más, donde se equivocó menos y la importancia que tienen los errores).

In [ ]:
print(classification_report(target_test, prediction_ht_test, target_names=target_names))
print('Optimización alcanzada de la función de pérdida:', ht_model.named_steps['mlpclassifier'].loss_)

In [ ]:
print(classification_report(target_test, prediction_relu_test, target_names=target_names))
print('Optimización alcanzada de la función de pérdida:', relu_model.named_steps['mlpclassifier'].loss_)

Al comparar los tiempos de ejecución de cada uno de los modelos se puede notar que el modelo con la función de activación hiperbólica tangencial tardo aproximadamente 2s y el modelo con la función de activación ReLU tardó apriximadamente 315ms. Es interesante notar, además de que la diferencia de tiempo de ejecución de ambos modelos es significativa, que el modelo que tiene menor cantidad de capas ocultas es el que se tarda más. Esto se podría explicar por la función de optimización utilizada para la asignación de los pesos, ya que en el caso de el modelo con función de activación hipérbolica, se utlizó el *solver adam* el cual para conjuntos de datos relativamente largos trabaja bastante rápido, sin embargo, si el conjunto de datos es pequeño, el *solver lbfgs* converge de manera más rápida con un mejor performance. Esto también se puede notar en la cantidad máxima de iteraciones de cada modelo, ya que para el primero fue necesario un máximo de 1000 iteraciones para que pudiera converger, a diferencia del segundo, que con 700 iteraciones máximas alcanzó a converger. 

Por otro lado, si se toma como medida de efectividad o de comparación entre los modelos el valor de *accuracy* el modelo 1 es el mejor modelo, ya que como se puede observar el modelo 1 obtuvo un *accuracy* de 99% en comparación con el modelo 2, el cual obtuvo un *accuracy* de 98%, sin embargo, como ya se ha mencionado anteriormente el valor de *accuracy* no siempre suele ser una buena opción para la comparación entre modelos, en especial si se cuenta con un conjunto de datos desbalanceado, por lo tal, si se utliza el *f1-score macro* se tiene que el mejor modelo es el modelo 2, con un *macro avg* de 98% en comparación con un 97% del modelo 1.  

Los errores encontrados en las matrices de clasificación mostradas anteriormente, a parte de permitir al modelo ir aprendiendo, también permiten establecer un punto de optimización del problema que se haya planteado. Ya que los errores permiten observar si el modelo, además de estar haciendo clasificación muy exactas o *accurate* permite también ver la cantidad de falsos positivos y negativos que el modelo esté haciendo. Estos errores permiten analizar y guiar al algoritmo para que se optimice de acorde a la métrica que más se ajusta a los requisitos del problema.

Como bien se mencionó en el apartado anterior, las equivocaciones del modelo están relacionadas con la precisión y la sensibilidad del mismo, en los reportes de clasificación se puede observar, como a pesar de que al clasificar las casas caras el algoritmo es bastante "sensible" o certero, no cuenta con una precisión de 1 debido a que tiene predicciones falsas positivas (las casas clasificadas como caras que no son caras). Por lo tal, la comparación y sobre todo la decisión de escoger el mejor algoritmo de clasificación depende del problema que se busca resolver o cuál es la métrica que se desea optimizar en las predicciones. Sin embargo, en el caso de redes neuronales y en la mayoría de algoritos de predicción, como bien se sabe, el objetivo principal es la optimización de una función de pérdida, la cual, en este contexto, puede establecer otra métrica u otro criterio para la decisión del mejor modelo, en este caso, el modelo 1 obtuvo una optimización final de 0.206 comparado con el modelo 2 el cual obtuvo una optimización final de 5.05-e05. Si bien el modelo 2 alcanzó una optimización mayor, es decir redujo más el error, esto podría representar un *overfitting* de los datos, ya que la función se está ajustando tanto a los datos de entrenamiento que pueda ser que en un futuro no sea capaz de clasificar nuevos datos. Por lo ta, se escoge como mejor modelo, en función de la optimización alcanzada y la capacidad de poder predecir con el menor número de falsos negativos como el modelo 1, o el modelo que usó la función de activació hiperbólica.  